# 5. COBRA vs. MASS 

When determining the discrepencies between COBRA and MASS, we have to understand the nuances and certain discrepencies between cobrapy and masspy. This documentation compares and contrasts the differences between the two models and discerns how mass action differs from flux balance analysis. We will start at the very begininning, from the getting started documentation and clarify the differences between cobrapy and masspy as we go through each documentation workbook. 

<font color='red'>

* Nice work, I like where you are going with this.

* Instead of using model_c and model_m, use the notations cobra_model and mass_model, cobra_reaction and mass_reaction, etc. (Use prefix cobra_ and mass_ when necessary)
    
* Leave a section for Genes, but skip making any section content for the time being

</font>

## 5.1 Inspection 

### 5.1.1 Importing Model Systems

At the beginning or each workflow, we import the speicifc methods that we wish to utilize for data analysis. For __cobrapy__, usual methodology to import the model and it's corresponding objects within the work environment can be accomplish through the following, 

In [4]:
from __future__ import print_function
from cobra import Model, Reaction, Metabolite
import cobra.test

However, __masspy__ has a slightly different syntax and requires more than just the importing of MASS itself; rather, it needs all of the objects which we utilize, from building models to creations of simulations to visualizing solutions,

In [8]:
from mass import (
    MassModel, MassMetabolite, MassReaction, Simulation, Solution,
    plot_simulation, plot_phase_portrait, plot_tiled_phase_portrait)
from mass.test import create_test_model

Other packages which are vital for information and visualization of data are also imported seperately, but is not seen to be the case for cobrapy, 

In [9]:
import numpy as np
import pandas as pd
import sympy as sym
import matplotlib as mpl
import matplotlib.pyplot as plt

Most of this comes down to the amount of information masspy can derive versus the amount of information cobrapy utilizes. From the basis of cobrapy, COBRA uses constraint based analysis to determine fluxes within and out of a particular system; for such model construction, an immense ammount of information is not required to derive constraints on a system based on it's fluxes. However, masspy requires much more adept analysis as an influx of information is created to determine the mass action stoichiometric solutions of any given model. Whereas COBRA focuses more on the mathematical aspect, MASS constructs models and determines the analytical portion of the model.

### 5.1.2 General breakdown of class, attributes and objects

For this type of analysis, we will create two test models; one is model_c and the other is model_m for cobrapy and masspy models respectively.

In [10]:
model_c = cobra.test.create_test_model("textbook")
model_m = create_test_model("textbook")

In [11]:
model_c

Name,e_coli_core
Memory address,0x0120438f60
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
Compartments,"cytosol, extracellular"


In [12]:
model_m

Name,Core_RBC_Model
Memory address,0x01207dacf8
Stoichiometric Matrix,48x53
Matrix Rank,44
Matrix Type,"dense, float64"
Number of Metabolites,48
Number of Reactions,53
Number of Initial Conditions,48
Number of Forward Rate Constants,53
Number of Equilibrium Constants,53
Number of Irreversible Reactions,11


More attributes are present within the masspy model, further qualifying the analysis of mass and cobra at the end of 5.1.1. MASS provides more flexibility for determining how the system behaves and allows for control over users to modulate different behaviors and perturbations whereas COBRA allows for the dimensional breakdown of the specific system through constraint based analysis and determination/ optimization of values based on the flux of reactions & metabolites into and out of the system. 

Upon further analysis, when we pull up a specific reaction, we can observe, 

In [13]:
model_m.reactions[29]

Reaction identifier,DPGM
Name,Diphosphoglyceromutase
Memory address,0x01207e8780
Subsystem,Hemoglobin
Stoichiometry,"_13dpg_c <=> _23dpg_c + h_c 3-Phospho-D-glyceroyl phosphate <=> 2,3-Disphospho-D-glycerate + H+"
GPR,
Kinetic Reversibility,True


In [14]:
model_c.reactions[29]

Reaction identifier,EX_glu__L_e
Name,L-Glutamate exchange
Memory address,0x0120950c88
Stoichiometry,glu__L_e --> L-Glutamate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Within __cobra__, we can distinctly see that an emphasis on lower and upper bounds is placed so as to constrain the reaction. This allows for mathematical calculations to be formulated as the fluxes within and out of a system can be based on a certain amount of values which allows for determining whether a reaction is pushing forward or going back *need to make this sound a bit more normal*

### 5.1.3 Genes